In [ ]:
# 학습 데이터 클래스 불균형
# 배는 colour data 없음 -> 뺄지 넣을지 선택

# 1. 학습데이터: 사과배복숭아_학습.xlsx
# 2. 알고리즘 LogisticClassification, SGDClassifier 중 선택
# 3. 기본 모델 검증과 교차 검증 비교 평가
# 4. GridSearchCV 클래스 이용하여 최적의 하이퍼파라미터 찾기
# 5. 제공된 테스트데이터(사과배복숭아_테스트.xlsx)로 기본 모델과 GridSearchCV 모델(best_estimator)와 성능 비교
# 6. 과제 보고서 작성(워드문서(hwp))
# - 코드와 각 셀의 실행 결과(텍스트 셀 포함)를 화면 캡처하여 붙이기
#   *** 내용 확인할 수 있는 크기로 화면 캡처할 것
# - 선택한 알고리즘의 성능 최적화 과정에 대한 실험 내용 작성

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import cross_validate,StratifiedKFold,GridSearchCV

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
apple_pear_peach = pd.read_excel('/content/drive/MyDrive/AIML_bum/사과배복숭아-학습.xlsx')

test = pd.read_excel('/content/drive/MyDrive/AIML_bum/사과배복숭아-테스트.xlsx')

In [61]:
apple_pear_peach.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fruit     168 non-null    object 
 1   weight    168 non-null    float64
 2   height    168 non-null    float64
 3   width     168 non-null    float64
 4   hardness  168 non-null    float64
 5   sweet     168 non-null    float64
 6   sour      168 non-null    float64
 7   color     119 non-null    float64
dtypes: float64(7), object(1)
memory usage: 10.6+ KB


In [62]:
#결측치가 존재하는 데이터를 학습 데이터에서 제외한다

train_input = apple_pear_peach[['weight',"height", "width","hardness","sweet","sour"]].to_numpy()
train_target = apple_pear_peach["fruit"].to_numpy()

test_input = test[['weight',"height", "width","hardness","sweet","sour"]].to_numpy()
test_target = test["fruit"].to_numpy()

In [63]:
print(train_input.shape,train_target.shape)

ss = StandardScaler()
ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)


(168, 6) (168,)


In [219]:
# 모델 생성
sc_ = SGDClassifier(loss="log_loss",max_iter=100,random_state=42)
sc_.fit(train_scaled, train_target)


SGDClassifier(loss='log_loss', max_iter=100, random_state=42)

In [220]:
# 기본 모델 검증
print(sc_.score(train_scaled,train_target))

print(sc_.score(test_scaled,test_target))

#과대적합의 경향을 보인다.


0.8214285714285714
0.5


In [206]:


splitter = StratifiedKFold(n_splits=3, shuffle=True) #(클래스 불균형을 고려하여 교차검증한다.)

cv2_results = cross_validate(estimator= sc_,
                            cv=   splitter,
                            X=  train_scaled,
                            y=  train_target,
                            return_train_score= True  )

cv2_results

# 과대적합의 경향성 뿐아니라 데이터를 어떻게 분할하고 교차검증하는지에 따라 점수가 크게 달라지므로 모델이 데이터에 의존적이라고 할 수 있다.

{'fit_time': array([0.00995469, 0.00667   , 0.00545907]),
 'score_time': array([0.00148654, 0.00091791, 0.00087738]),
 'test_score': array([0.76785714, 0.85714286, 0.89285714]),
 'train_score': array([0.91964286, 0.88392857, 0.94642857])}

In [227]:
# gridsearchCV를 통해 과대 적합을 해결하기위한 규제 상수를 찾는다.
model = SGDClassifier(loss="log_loss",max_iter=100,random_state=42,learning_rate='adaptive',eta0=5.0)
alpha_list = [1 / (10 ** i) for i in range(10)]
params = {'alpha': alpha_list }
splitter = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

gs = GridSearchCV(estimator = model , param_grid= params, n_jobs = 1, cv = splitter)


gs.fit(train_scaled, train_target)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
             estimator=SGDClassifier(eta0=5.0, learning_rate='adaptive',
                                     loss='log_loss', max_iter=100,
                                     random_state=42),
             n_jobs=1,
             param_grid={'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06,
                                   1e-07, 1e-08, 1e-09]})

In [228]:
print(gs.best_score_)


0.875


In [229]:
gs.best_params_

{'alpha': 1e-05}

In [230]:
print(gs.best_estimator_)

print(sc_)

SGDClassifier(alpha=1e-05, eta0=5.0, learning_rate='adaptive', loss='log_loss',
              max_iter=100, random_state=42)
SGDClassifier(loss='log_loss', max_iter=100, random_state=42)


In [231]:
print(gs.best_estimator_.score(train_scaled,train_target))
print(gs.best_estimator_.score(test_scaled,test_target))

print(sc_.score(train_scaled,train_target))
print(sc_.score(test_scaled,test_target))

0.8809523809523809
0.7
0.8214285714285714
0.5


In [232]:
cv2_results = cross_validate(estimator= gs.best_estimator_,
                            cv=   splitter,
                            X=  train_scaled,
                            y=  train_target,
                            return_train_score= True  )
cv2_results

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


{'fit_time': array([0.01736045, 0.01214933, 0.01043844]),
 'score_time': array([0.00176001, 0.00212884, 0.00159335]),
 'test_score': array([0.85714286, 0.83928571, 0.92857143]),
 'train_score': array([0.89285714, 0.89285714, 0.94642857])}


##*summary*
#기본 모델에서 학습률을 5.0으로 설정해서 빠르게 가중치를 찾을수 있었고
#grid_search를 통해서 경사하강법 모델의 최적의 규제 상수를 찾을수 있었다.

#이를 통해서 과대적합 문제를 해소하고, 전반적인 모델의 성능을 올릴 수 있었다.